In [ ]:
#!pip install pydrive

In [2]:
import os

In [3]:
contenido = os.listdir(r'D:\PYTHON\utea_scripts\reportes_credito_siembra\pdfs')

dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
dic_files

{11641: '11641_ROJAS SANTISTEVAN JUAN MANUEL .pdf',
 17683: '17683_ROJAS MARINA SANTISTEVAN DE .pdf',
 18358: '18358_ROJAS AMELUNGE CARLOS .pdf',
 1988: '1988_CLAROS CADIMA JHONNY.pdf',
 2794: '2794_DELGADILLO RODRIGUEZ JUAN CARLOS.pdf',
 388: '388_AGROPECUARIA MARIANA S.R.L. .pdf',
 41534: '41534_CAMPBELL MEDINA FERNANDO WALTER.pdf',
 842: '842_AGRODAR S.R.L..pdf',
 8624: '8624_MORENO TORRICO JOSE ALBERTO.pdf'}

#  CARGAR FILES A DRIVE

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [5]:
print(len(dic_files), 'INFORMES')
dic_files

9 INFORMES


{11641: '11641_ROJAS SANTISTEVAN JUAN MANUEL .pdf',
 17683: '17683_ROJAS MARINA SANTISTEVAN DE .pdf',
 18358: '18358_ROJAS AMELUNGE CARLOS .pdf',
 1988: '1988_CLAROS CADIMA JHONNY.pdf',
 2794: '2794_DELGADILLO RODRIGUEZ JUAN CARLOS.pdf',
 388: '388_AGROPECUARIA MARIANA S.R.L. .pdf',
 41534: '41534_CAMPBELL MEDINA FERNANDO WALTER.pdf',
 842: '842_AGRODAR S.R.L..pdf',
 8624: '8624_MORENO TORRICO JOSE ALBERTO.pdf'}

In [6]:
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "1RA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(r'D:\PYTHON\utea_scripts\reportes_credito_siembra\pdfs//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 11641_ROJAS SANTISTEVAN JUAN MANUEL .pdf with mimeType application/pdf
Created file 17683_ROJAS MARINA SANTISTEVAN DE .pdf with mimeType application/pdf
Created file 18358_ROJAS AMELUNGE CARLOS .pdf with mimeType application/pdf
Created file 1988_CLAROS CADIMA JHONNY.pdf with mimeType application/pdf
Created file 2794_DELGADILLO RODRIGUEZ JUAN CARLOS.pdf with mimeType application/pdf
Created file 388_AGROPECUARIA MARIANA S.R.L. .pdf with mimeType application/pdf
Created file 41534_CAMPBELL MEDINA FERNANDO WALTER.pdf with mimeType application/pdf
Created file 842_AGRODAR S.R.L..pdf with mimeType application/pdf
Created file 8624_MORENO TORRICO JOSE ALBERTO.pdf with mimeType application/pdf
end


In [7]:
print(len(dic_urls), 'URLs')
dic_urls

9 URLs


{11641: 'https://drive.google.com/file/d/1ol4LtPVbPwbK5GipVEmrDvZGW8vTgRl6/view?usp=drivesdk',
 17683: 'https://drive.google.com/file/d/1bUX1J-0UZ92Xxu70UusclBl2pYT0MKoX/view?usp=drivesdk',
 18358: 'https://drive.google.com/file/d/1Cr0dWRsL2IDEw9-pDB5YSy7wM1UH-5Yj/view?usp=drivesdk',
 1988: 'https://drive.google.com/file/d/1R6_kMkwNVrp7-9k5N8kNTNbMCPa1CwVP/view?usp=drivesdk',
 2794: 'https://drive.google.com/file/d/1hvFA93Ilg0elmCFRfd78dTzYItq6ZR-T/view?usp=drivesdk',
 388: 'https://drive.google.com/file/d/1fSx9h0SAv04xAK-nooOsysrF80IqscgW/view?usp=drivesdk',
 41534: 'https://drive.google.com/file/d/1uZvAbrSi1gbBlBvTuDLg36PDkjb6SoND/view?usp=drivesdk',
 842: 'https://drive.google.com/file/d/1aOdU6PqUGKlBdNnsD6EfLD92hEo9o8Q3/view?usp=drivesdk',
 8624: 'https://drive.google.com/file/d/1f17rVKPTrL-iU01BDYMRUG6G5eVYxyV4/view?usp=drivesdk'}

# CONECTAR A AMIGOCLOUD

In [9]:
from amigocloud import AmigoCloud

In [10]:
# CONECTAR A AMIGOCLOUD
acceso = open('../../../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [11]:
for key in dic_urls:
    query = {'query': 'update dataset_307560 set link_doc_1ra = \'{link}\' where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = \'{cod}\''.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
    print(key)
print('end')

11641
17683
18358
1988
2794
388
41534
842
8624
end


# -----------------------------------------------------------

# CONECCION A DRIVE

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

In [ ]:
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "REPORTES_CREDITO"):
      fileID = file['id']

file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
file1.SetContentFile("file.pdf")
file1.Upload() # Upload the file.
print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))

In [ ]:
permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})

print(file1['alternateLink'])  # Display the sharable link.

In [ ]:
for file in fileList[0]:
    print(file, '       ',fileList[0][file])

In [ ]:
fileList[0]['title']